# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

OPENAI_API_KEY=****here
LANGSMITH_API_KEY=****754b
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [3]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1:8b", temperature=0)

In [4]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model=llm,
    tools=mcp_tools,
    system_prompt=
        "You are a helpful assistant. "
        "IMPORTANT: When using time tools, you must use official IANA timezone names. "
        "For example, use 'America/Los_Angeles' for San Francisco or Seattle, "
        "and 'America/New_York' for NYC. Do not guess names like 'America/San_Francisco'."
)

Ask about the current time in San Francisco.

In [5]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in SF right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in SF right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (25b11d62-03c4-4358-946e-2660927cac37)
 Call ID: 25b11d62-03c4-4358-946e-2660927cac37
  Args:
    format: DateTime
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

2026-01-15 11:24:09
================================== Ai Message ==================================

The current time in San Francisco is January 15, 2026 at 11:24 AM.


In [6]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in Seattle right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in Seattle right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (fc5e6e04-baf2-40ae-b36a-cc7cc4976cd1)
 Call ID: fc5e6e04-baf2-40ae-b36a-cc7cc4976cd1
  Args:
    format: DateTime
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

2026-01-15 11:28:37
================================== Ai Message ==================================

The current time in Seattle is January 15, 2026 at 11:28:37.


In [7]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in NY right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in NY right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (d6cdbc59-9487-491f-9b02-926b9adf6827)
 Call ID: d6cdbc59-9487-491f-9b02-926b9adf6827
  Args:
    format: DateTime
    timezone: America/New_York
================================= Tool Message =================================
Name: current_time

2026-01-15 14:33:01
================================== Ai Message ==================================

The current time in New York is 2:33 PM on January 15, 2024.
